In [1]:
from pathlib import Path
import pandas as pd
from typing import List

In [2]:
base_path = Path("/workspace/navsim_workspace/exp")
csv_paths = list(base_path.rglob("*.csv"))
csv_paths = [p for p in csv_paths if "eval" in str(p)]
if not csv_paths:
    raise ValueError("No CSV files found in the specified directory.")
csv_paths

[PosixPath('/workspace/navsim_workspace/exp/diffusiondrive_agent_original_public_eval/2025.06.18.06.48.59/2025.06.18.06.58.56.csv'),
 PosixPath('/workspace/navsim_workspace/exp/eval_diffusiondrive_agent_testOnNewTorchVersion_bs128_ep400_epoch=39-step=3360/2025.07.07.11.33.18/2025.07.07.11.43.29.csv'),
 PosixPath('/workspace/navsim_workspace/exp/eval_diffusiondrive_agent_testOnNewTorchVersion_bs128_ep400_epoch=49-step=4200/2025.07.07.11.43.45/2025.07.07.11.53.57.csv'),
 PosixPath('/workspace/navsim_workspace/exp/eval_diffusiondrive_agent_testOnNewTorchVersion_bs256_ep400_epoch=99-step=4200/2025.07.07.12.56.28/2025.07.07.13.06.35.csv'),
 PosixPath('/workspace/navsim_workspace/exp/eval_diffusiondrive_agent_testOnNewTorchVersion_bs32_ep400_epoch=49-step=16650/2025.07.07.13.38.04/2025.07.07.13.48.13.csv'),
 PosixPath('/workspace/navsim_workspace/exp/eval_diffusiondrive_agent_testOnNewTorchVersion_bs32_ep400_epoch=29-step=9990/2025.07.07.13.17.16/2025.07.07.13.27.17.csv'),
 PosixPath('/works

In [3]:
def compare_result(paths: List[Path]):
    df_lst = []
    names_lst = []
    for path in paths:
        df_lst.append(pd.read_csv(path).iloc[-1:,3:])
        names_lst.append(path.parents[1].name)
    df = pd.concat(df_lst, keys=names_lst)
    df = df.reset_index(names=["model", "index"]).drop(columns=["index"])
    
    # Extract epoch from model name
    epoch_col = df["model"].str.extract(r'epoch=(\d+)')
    df.insert(1, "epoch", epoch_col)
    # The default value for epoch is 99
    df["epoch"] = df["epoch"].fillna(99)
    df["epoch"] = df["epoch"].astype(int)
    
    # Extract batch size from model name
    batch_size_col = df["model"].str.extract(r'bs(\d+)')
    df.insert(2, "batch_size", batch_size_col)
    # The default value for batch size is 64
    df["batch_size"] = df["batch_size"].fillna(64)
    df["batch_size"] = df["batch_size"].astype(int)
    
    df["model"] = df["model"].str.replace("eval_diffusiondrive_agent_default_", '')
    df["model"] = df["model"].str.replace("diffusiondrive_agent_", '')

    df = df.sort_values(by=["score", "epoch", "batch_size"], ascending=[False, True, True])

    
    return df.style.highlight_max(color='purple', axis=0)
compare_result(csv_paths)

,model,epoch,batch_size,no_at_fault_collisions,drivable_area_compliance,ego_progress,time_to_collision_within_bound,comfort,driving_direction_compliance,score
6,bs32ep1000_epoch=999-step=333000,999,32,0.980858,0.966820,0.826374,0.942615,0.999918,0.980076,0.882754
7,bs32ep1000_epoch=999-step=333000,999,32,0.981228,0.966573,0.826420,0.942450,0.999918,0.980323,0.882564
14,eval_testOnNewTorchVersion_bs32_ep400_epoch=59-step=19980,59,32,0.982628,0.964021,0.823108,0.947966,1.000000,0.980776,0.881928
36,ep300_epoch=299-step=50100,299,64,0.981064,0.964844,0.823276,0.944426,0.999918,0.980487,0.881188
35,ep300_epoch=299-step=50100,299,64,0.981064,0.964515,0.822941,0.944591,0.999918,0.980693,0.880890
0,original_public_eval,99,64,0.982175,0.962374,0.822096,0.947472,0.999835,0.980899,0.880551
30,bs32ep300_epoch=49-step=16650,49,32,0.982710,0.963445,0.825526,0.939239,0.999918,0.981764,0.879374
13,eval_testOnNewTorchVersion_bs32_ep400_epoch=199-step=66600,199,32,0.982175,0.962868,0.819429,0.945908,0.999918,0.980817,0.879201
4,eval_testOnNewTorchVersion_bs32_ep400_epoch=49-step=16650,49,32,0.981928,0.960481,0.823709,0.945002,1.000000,0.981023,0.879137
31,bs32ep300_epoch=49-step=16650,49,32,0.982669,0.962786,0.824923,0.939322,1.000000,0.981681,0.878875
